In [1]:
import asyncio
import collections

import aiohttp
from aiohttp import web
import tqdm

%run flags2_common.ipynb import main, HTTPStatus, Result, save_flag

In [2]:
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

In [3]:
class FetchError(Exception):
    def __init__(self, country_code):
        self.country_code = country_code

In [4]:
@asyncio.coroutine
def http_get(url):
    res = yield from aiohttp.request('GET', url)
    if res.status == 200:
        ctype = res.headers.get('Content-type', '').lower()
        if 'json' in ctype or url.endswith('json'):
            data = yield from res.json()  # <1>
        else:
            data = yield from res.read()  # <2>
        return data

    elif res.status == 404:
        raise web.HTTPNotFound()
    else:
        raise aiohttp.errors.HttpProcessingError(
            code=res.status, message=res.reason,
            headers=res.headers)


C:\Users\Ivo\AppData\Local\Temp\ipykernel_12628\422895157.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def http_get(url):


In [5]:
@asyncio.coroutine
def get_country(base_url, cc):
    url = '{}/{cc}/metadata.json'.format(base_url, cc=cc.lower())
    metadata = yield from http_get(url)  # <3>
    return metadata['country']

C:\Users\Ivo\AppData\Local\Temp\ipykernel_12628\4263828679.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def get_country(base_url, cc):


In [6]:
@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    return (yield from http_get(url)) # <4>

C:\Users\Ivo\AppData\Local\Temp\ipykernel_12628\4078728701.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def get_flag(base_url, cc):


In [7]:
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore): # <5>
            image = yield from get_flag(base_url, cc)
        with (yield from semaphore):
            country = yield from get_country(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        country = country.replace(' ', '_')
        filename = '{}-{}.gif'.format(country, cc)
        loop = asyncio.get_event_loop()
        loop.run_in_executor(None, save_flag, image, filename)
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)

C:\Users\Ivo\AppData\Local\Temp\ipykernel_12628\3710002885.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def download_one(cc, base_url, semaphore, verbose):


In [8]:
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)
    to_do = [download_one(cc, base_url, semaphore, verbose)
             for cc in sorted(cc_list)]

    to_do_iter = asyncio.as_completed(to_do)
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))
    for future in to_do_iter:
        try:
            res = yield from future
        except FetchError as exc:
            country_code = exc.country_code
            try:
                error_msg = exc.__cause__.args[0]
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status

        counter[status] += 1

    return 

C:\Users\Ivo\AppData\Local\Temp\ipykernel_12628\2807612423.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def downloader_coro(cc_list, base_url, verbose, concur_req):


In [9]:
def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)
    loop.close()

    return counts

In [10]:
if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

usage: ipykernel_launcher.py [-h] [-a] [-e] [-l N] [-m CONCURRENT] [-s LABEL] [-v] [CC ...]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\Ivo\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
